# PHYS 462 Computational Project - Fermi Gas in a Box
### Author: Ansh Gupta

## Project Description:
Fermions are particles with a half-integer spin, like $\frac{1}{2}$, $\frac{3}{2}$, etc. Unlike bosons, they obey the Pauli exclusion principle, meaning that two fermions cannot occupy the same quantum state.